<a href="https://colab.research.google.com/github/afaq5587/Lang-chain/blob/main/Rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

!pip install tqdm
!pip install -qU pypdf
!pip install langchain
!pip install langchain-google-genai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.2 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
userdata.get("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hello world")

from google.colab import drive
drive.mount('/content/drive')

PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

Mounted at /content/drive


In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

index_name = "start1"
pc = Pinecone(api_key= PINECONE_API_KEY)
# Check if the index already exists
indexes = pc.list_indexes()  # This will return a list of existing indexes

if index_name in indexes:
    print(f"Index '{index_name}' already exists. Choose a different name.")
else:
    # Proceed with creating the index
    pc.create_index(
        name=index_name,
        dimension=768,  # Example dimension for 'embedding-001'
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
index = pc.Index(index_name)
vector_store = PineconeVectorStore(embedding=embeddings, index=index, text_key="text")


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = TextLoader("documents.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'documents.txt'}, page_content='hi how are you')]


In [ ]:

from tqdm import tqdm

# Create embeddings and upload to Pinecone
vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    # Generate a unique ID for each document (e.g., UUID or hash)
    doc_id = str(hash(doc.page_content))
    # Upsert the vector with a unique ID
    metadata = {
    "text": doc.page_content,
    }
    # Upsert the vector with metadata
    index.upsert(vectors=[(doc_id, vector, metadata)])


100%|██████████| 1/1 [00:00<00:00,  1.48it/s]


In [ ]:
from langchain.vectorstores import Pinecone


retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


# Convert it into a retriever compatible with RetrievalQA
retriever = vector_store.as_retriever()


<ipython-input-14-fe5b3ffb0510>:4: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",  # Other options: "map_reduce", "refine"
    retriever=retriever
)

In [ ]:

question = input("I am a langchain chatbot. you can ask me for abdul qadeer khan.\n ")
response = qa_chain.run(question)
print(response)


I am a langchain chatbot. you can ask me for abdul qadeer khan.
 whell done
I don't know.

